# Initial Closed Form Design

My goal is to create a V-tail to print for the Stratosurfer. First thing should be to take a look at the Stratosurfer's wing dimensions in order to start outlining the requirements for the tail section.

I will start out by trying to find some measurements of the Stratosurfer. What I can't measure, I will assume. A good place to start is [Mark Qvale's website](http://www.itsqv.com/QVM/index.php?title=Strix_StratoSurfer), where he graciously provides some dimensions. I'll list them here in code and have them saved to a file for future use.

In [7]:
itsqv_config = {
    "b_m": 1.500, # Wingspan, meters
    "S_m": 0.0274, # Wing area, meters squared
    "length_m": 1.000, # Fuse length, meters
    "wieght_kg": 1.800, # Assumed weight
    "cg_wing_m": 0.0675, # Center of gravity from wing root
}

import os
import yaml

os.makedirs("data", exist_ok=True)
with open("data/itsqv-dimensions.yaml", "w") as fid:
    yaml.dump(itsqv_config, fid)

Here we have the dimensions of the wing, but we don't have information about the lift-slope curve $\partial C_L / \partial \alpha_w$, which we need in order to design the tail. I measured the cross section of the wing by wrapping the wing with paper-mache, letting it dry, and tracing the interior. I used my [airfoil plotter](https://github.com/MattMarti/airfoil-plotter) to plot out the diagram and approximate it with a NACA airfoil. I found that it looked approximately like a **NACA 4410** airfoil. Unfortunately, the [NACA airfoil database](http://airfoiltools.com/search/index?m%5Bgrp%5D=naca4d&m%5Bsort%5D=1&MAirfoilSearchForm%5BtextSearch%5D=5408&MAirfoilSearchForm%5BmaxThickness%5D=&MAirfoilSearchForm%5BminThickness%5D=&MAirfoilSearchForm%5BmaxCamber%5D=&MAirfoilSearchForm%5BminCamber%5D=&MAirfoilSearchForm%5Bgrp%5D=naca4d&MAirfoilSearchForm%5Bsort%5D=1&yt0=Search) doesn't have this airfoil. However, I can use [xfoil](https://web.mit.edu/drela/Public/web/xfoil/) to try to get information about the coefficient of lift.

## XFOIL Wing Analysis

I will use [XFOIL](https://web.mit.edu/drela/Public/web/xfoil/) to try to get the coefficient of lift and drag (as a function of angle of attack) from the airfoil of the main wing. These parameters will be important to have to design the tail section.


I'll document my inputs to the program here for future reference:

1. `NACA 4410` - Set the airfoil to the 4-digit **4410** since that's about what the airfoil looks like
1. `PPAR` - Open the options menu
1. `N 250` - Set the number of panel nodes to 250. Seems pretty high, and the array limit in my version of XFOIL is 494, so I can't use 500. I'll check out whether decreasing the number of points causes it to converge to different results and report on it anecdotally.
1. `<cr>` - Hit enter to exit `PPAR` menu.
1. `<cr>` - Hit enter again to actually exit `PPAR` menu.
1. `oper` - Open the "Direct Operating Points" menu.
1. `visc 1` - Toggle viscous mode on
1. `mach  0.0669` - Set Mach Number. See below for details on how I got this number.
1. `re 308639` - Set Reynold's Number. See below for details on how I got this number.



### Mach Number

I need to make some assumptions about where I'll be flying. I live at an altitude of about 650 m, but I plan to travel to fly. I'll do my analysis for 1000 m above sea level, but I'd like to return to study the flight envelope at maybe 3000 m in case I decide to go somewhere cool in Colorado. Legally, I can't fly above 400 ft, but I live in a mountainous area so it'd be good to have a little flexibility after the analysis.



For speed, let's assume 50 mph, or about 22.5 m/s. It's a rough conversion, but hey, it's an assumption anyway. Eventually I need to take a look at what the stall speed is, and the max speed. Here's the stats:

- $v_\infty$ = 22.5 [m/s]
- $h_{alt}$ = 1000 [m/s]


Looking up in the standard atmosphere table \[Anderson J., Introduction to Flight, eighth edition, 2016\], I find that the freestream temperature, pressure, and density are:


- $T_\infty$ = 281.66 [K]
- $p_\infty$ = 8.9876e4 [N/m^2]
- $\rho_\infty$ = 1.1117 [kg/m^3]


Then the Mach number is:


$ M_\infty = \frac{v_\infty}{\sqrt{\gamma R T_\infty}} $


And it's air, so the ratio of specific heats is $1.4$, and the gas constant is $R = 287.05 [J/(kg K)]$

In [6]:
import numpy as np

gamma = 1.4
R = 287.05
T_inf_K = 281.66
v_inf_ms = 22.5

M_inf = v_inf_ms / np.sqrt(1.4 * R * T_inf_K)
print(f"Mach number: {M_inf:.4f}")

Mach number: 0.0669


### Reyonld's Number

I'm going to have to make a couple of assumptions here as well. I can refer to my measurements of the airfoil from earlier. The equation for Reynold's Number of an airfoil is:


$ Re = \frac{\rho_\infty v_\infty c}{\mu} $


We can [look up the dynamic viscocity](https://www.engineersedge.com/physics/viscosity_of_air_dynamic_and_kinematic_14483.htm#:~:text=The%20viscosity%20of%20air%20depends,10%2D5%20Pa%C2%B7s%20.) of air and find $\mu = 1.778e-5$ (at $10^\circ C$). We already have the density and freestream velocity. The chord length $c$ was measured when I made the paper mache wrap, and on the Stratosurfer is 215 mm. However, the airfoil plotter tool shows that 215 mm is about 98% of the total airfoil length because the ends don't meet in the back.

In [10]:
rho_inf_si = 1.1117 # Whatever the SI units are
mu = 1.778e-5

c_m = 0.215 / 0.98

Re = rho_inf_si * v_inf_ms * c_m / mu
print(f"Reynold's Number: {Re:.0f}")

Reynold's Number: 308639
